In [176]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape

from ASCE import load_combos
from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
import concrete_design.beam_calculations.design_properties as bm
conc_density = 150.0 #pcf

<div style="page-break-after: always;"></div>

# Concrete Beam Design
## Inputs

In [177]:
# Materials
fc = 3000 #psi
fy = 60000 #psi
fyt = 60000 #psi
Es = 29000000 #psi

# Dimensions
b = 18 #in
h = 18 #in
ln = 10*12 #in
cover_t = 1.5 #in
cover_b = 1.5 #in
cover_side = 1.5 #in

#### Deep beam check

- span does not exceed 4 * h

- concentrated load exists within a distance 2h from face of support

In [178]:
if ln < 4*h:
    print("DEEP BEAM")
else:
    print("beam theory OK")

beam theory OK


### Loads

In [179]:
SW = conc_density * b * h / 12**2 #plf
DL = 15 #psf
LL = 40 #psf

LN = ln / 12 #ft
trib = 15 #ft
D = DL * trib / 12 + SW #plf
L = LL * trib / 12 #plf
w_u = load_combos.LRFD(D, L) #plf

Mu = w_u * LN**2 / 8 / 1000 #kip-ft
Vu = w_u * LN / 2 / 1000 #kips
Tu = 4 * LN / 2 #kip-ft
print(f"Mu = {round(Mu, 2)} kip-ft")
print(f"Vu = {round(Vu, 2)} kips")
print(f"Tu = {round(Tu, 2)} kip-ft/ft")

Mu = 6.34 kip-ft
Vu = 2.54 kips
Tu = 20.0 kip-ft/ft


### Materials and Geometry

In [180]:
# Shear Stirrups
stirrup_size = "#4"
num_legs = 2
s = 12 #in O.C.

db_tie = rebar.loc[stirrup_size, "Diameter"]
Av_placed = num_legs * rebar.loc[stirrup_size, "Area"]
f"Av = {round(Av_placed, 2)} in^2"

'Av = 0.4 in^2'

In [181]:
# Tension Face Longitudinal Reinforcement
bar = "#9"
Ab_t = rebar.loc[bar, "Area"]
db_t = rebar.loc[bar, "Diameter"]
n = 4 #number of bars
layers = 1

d_t = h - cover_b - db_tie - db_t/2
print(f"d_t = {round(d_t, 3)} in")

As_t_placed = n * Ab_t #in^2
print(f"As = {round(As_t_placed, 3)} in^2")

d_t = 15.436 in
As = 4.0 in^2


In [182]:
# Compression Face Longitudinal Reinforcement
bar = "#9"
Ab_c = rebar.loc[bar, "Area"]
db_c = rebar.loc[bar, "Diameter"]
n = 4 #number of bars
layers = 1

d_c = cover_t + db_tie + db_c/2
print(f"d_c = {round(d_c, 3)} in")

As_c_placed = n * Ab_c #compression reinforcement
print(f"As = {round(As_c_placed, 3)} in^2")

d_c = 2.564 in
As = 4.0 in^2


In [183]:
# Percentage of reinforcement reserved for Torsion resistance
percent_long_for_torsion = 25 #%
percent_ties_for_torsion = 50 #%

## Strength Design

### $\phi$ Mn $\geq$ Mu

In [184]:
include_compression_steel = "Yes"
if include_compression_steel == "No":
    As_c = 0

print(As_c)

4.0


In [185]:
As_t = As_t_placed * (100-percent_long_for_torsion)/100 #in^2
As_c = As_c_placed * (100-percent_long_for_torsion)/100 #in^2

#### $\phi$ Factor Calculation

In [186]:
c = bm.compression_block(As_t, fy, Es, b, fc, Apr_s=As_t, dpr=d_c) 
a = bm.equiv_compr_block(c)
eps_t = bm.tension_steel_strain(d_t, c)
eps_cs = bm.compr_steel_strain(c, eps_t, d_t)
φ_b = bm.phi_factor(eps_t, fy, Es)

c, a, eps_t, eps_cs, φ_b

(3.2316043449791945, 2.7468636932323154, 0.011329724513443844, 0.003, 0.9)

#### Flexural Strength

In [187]:
fs = min(eps_cs * Es, fy) #psi (negative strain)

C_s = As_c * fs #lbs
C_c = As_t*fy - C_s #lbs
T = As_t * fy #lbs

Mn = T*d_t - C_c*a/2 - C_s*d_c #lb-in
φMn = φ_b * Mn

print(f"φ_b = {round(φ_b, 3)}")
print(f"Mu = {round(Mu, 2)} kip-ft")
print(f"Mn = {round(Mn / 12 / 1000, 2)} kip-ft")
print(f"φMn = {round(φMn / 12 / 1000, 2)} kip-ft")
f"DCR = {round(Mu / (φMn / 12 / 1000), 3)}"

φ_b = 0.9
Mu = 6.34 kip-ft
Mn = 193.08 kip-ft
φMn = 173.77 kip-ft


'DCR = 0.037'

In [188]:
Mn /1000, C_s/1000, C_c/1000

(2316.96, 180.0, 0.0)

### $\phi$ Vn $\geq$ Vu

In [189]:
Av = Av_placed * (100-percent_ties_for_torsion)/100 #in^2

#### Shear Strength

In [190]:
Vc = 2 * sqrt(fc) * b * d_t #lbs
f"Vc = {round(Vc, 2)} lbs"

'Vc = 30436.72 lbs'

In [191]:
Vs = Av * fyt * d_t / s #lbs
f"Vs = {round(Vs, 2)} lbs"

'Vs = 15436.0 lbs'

In [192]:
φ_v = 0.75
Vn = Vc + Vs
print(f"Vn = {round(Vn, 2)} lbs")
print(f"φVn = {round(φ_v * Vn, 2)} lbs")

Vn = 45872.72 lbs
φVn = 34404.54 lbs


In [193]:
f"DCR = {round(Vu / (φ_v * Vn / 1000), 3)}"

'DCR = 0.074'

### $\phi$ Tn $\geq$ Tu

- Nonprestressed member with no significant axial force

In [194]:
φ_tor = 0.75
A_cp = h * b #in^2
p_cp = 2*h + 2*b #in
T_th = sqrt(fc) * (A_cp**2 / p_cp) / 12 / 1000 #kip-ft
f"Threshold Torsion, T_th = {round(T_th, 3)}"

'Threshold Torsion, T_th = 6.655'

In [195]:
# ACI318-19 22.7.1.1 and 22.7.2.1
Tu < φ_tor * T_th, sqrt(fc) < 100 #psi

(False, True)

In [196]:
# Torsion Capacity Calculation

#area of section for shear flow path
A_oh = (h - cover_t - cover_b - db_tie) * (b - 2*cover_side - db_tie) #in^2
A_o = 0.85 * A_oh #in^2

#reinforcement used for Torsion
A_l = percent_long_for_torsion/100 * (As_t_placed + As_c_placed) #in^2
A_t = percent_ties_for_torsion/100 * Av_placed #in^2

#space truss strut angle from tie spacing and depth
strut_height = d_t - d_c #in
strut_length = s #in
θ = arctan(strut_height / strut_length) #rad

#perimeter of the centerline of the outermost stirrup
p_h = 2*(h - cover_t - cover_b - db_tie) + 2*(b - 2*cover_side - db_tie) #in

Tn = min(
    2 * A_o * A_t * fyt / tan(θ) / s,
    2 * A_o * A_l * fy * tan(θ) / p_h,
) #in-lbs

f"Tn = {round(Tn/12/1000, 3)} kip-ft"

'Tn = 27.768 kip-ft'

In [197]:
f"DCR = {round(Tu / (φ_tor * Tn / 12/1000), 3)}"

'DCR = 0.96'

### $\phi$ Pn $\geq$ Pu

## Deflection

### Short-term Deflection

In [198]:
w_a = load_combos.ASD(D, L)

Ma = w_a * LN**2 / 8 * 12 #lb-in
f"Ma = {round(Ma / 12, 2)} lb-ft"

'Ma = 5078.12 lb-ft'

In [199]:
fr = 7.5 * sqrt(fc) #psi
Ig = b * h**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in")
yt = h / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

Ig = 8748.0 in


'Mcr = 33274.15 lb-ft'

In [200]:
# Elastic Modulus
Ec = elastic_modulus(fc=fc) #psi
print(f"Ec = {round(Ec, 2)} ksi")
# Moment of Inertia
Ie = Ig
Icr = cracked_moment_inertia(b=b, d=d_t, Ec=Ec, As=As_t)
# Icr = cracked_moment_inertia(b=b, h=h, d=d_t, d_c=d_c, Ec=Ec, As=As_t, As_c=As_c)
print(f"Icr = {round(Icr, 2)} in^4")
if Ma > 2/3*Mcr:
    Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
    print(f"Ie = {round(Ie, 2)} in")
else:
    print("section uncracked")

#Elastic Deflection
defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * 1000 * Ie)
f"Elastic Deflection = {round(defl_elastic, 3)} in"

NameError: name 'elastic_modulus' is not defined

### Long-term Deflection

In [ ]:
# Deflection Factor
xi = 2.0 #duration factor
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 0.392'

In [ ]:
# Long-Term Deflection
defl_long = defl_elastic * lmbda_dlt #in
f"Long-Term Deflection = {round(defl_long, 3)} in"

'Long-Term Deflection = 0.0 in'

In [ ]:
# Total Deflection
defl_total = defl_elastic + defl_long
f"Total Deflection = {round(defl_total, 3)} in"

'Total Deflection = 0.0 in'

## Minimum Requirements

### Dimension Requirements

#### Minimum Beam Depth

In [ ]:
h_min = ln / 16
print(f"h_min (simply supported) = {round(h_min, 2)} in")

h_min = ln / 18.5
print(f"h_min (one end continuous) = {round(h_min, 2)} in")

h_min = ln / 21
print(f"h_min (both ends continuous) = {round(h_min, 2)} in")

h_min = ln / 8
print(f"h_min (cantilever) = {round(h_min, 2)} in")

h_min (simply supported) = 15.0 in
h_min (one end continuous) = 12.97 in
h_min (both ends continuous) = 11.43 in
h_min (cantilever) = 30.0 in


In [ ]:
Vu <= φ_v * (Vc + 8*sqrt(fc) * b * d_t)

True

### Minimum Reinforcement

#### Flexural Reinforcement

In [ ]:
As_min = max(3*sqrt(fc)/fy, 300/fy) * b * d_t #in^2

f"As,min = {round(As_min, 3)} in^2"

'As,min = 1.606 in^2'

#### Shear Reinforcement

In [ ]:
if Vu > φ_v*sqrt(fc)*b*d_t:
    Av_min = max(0.75*sqrt(fc), 50) * b / fyt #in^2
    print(f"Av,min = {round(Av_min, 3)} in^2")
else:
    Av_min = 0
    print("Shear reinforcement not required")

Shear reinforcement not required


#### Torsional Reinforcement